# Lecture Overview

- Dividend discount models
    - General formula
    - Constant growth
    - Two-stage growth
    - Multi-stage growth
    

- Data
    - Firm-level data
        - Past dividends
        - Payout ratios and profitability
    - Industry-level data
    
    
- Valuation exercises
    - Constant growth
    - Two-stage growth
    - Three-stage growth

In [1]:
# Import packages
import yfinance as yf
import pandas as pd
import numpy as np
import pandas_datareader as pdr
import statsmodels.api as sm

# Pretty print all cell's output and not just the last one
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# 1. Dividend discount models (DDM)

The intrinsic value of the firm's equity at time $t$ (call it $V_t$) is the discounted value of all (expected) future dividends:

$$V_t = \frac{D_{t+1}}{1+r} + \frac{D_{t+2}}{(1+r)^2} + \frac{D_{t+3}}{(1+r)^3} + ...$$        

or, equivalently

$$V_t = \frac{D_{t+1}}{1+r} + \frac{D_{t+2}}{(1+r)^2} + \frac{D_{t+3}}{(1+r)^3} + ... + \frac{D_{t+H} + P_{t+H}}{(1+r)^H}$$        

where $P_{t+H}$ (the price $H$ periods from now) is often called the "terminal value" of the stock. The discount rate $r$ (sometimes called the market capitalization rate) is the expected return on the stock (or, equivalently, the firm's cost of equity). As we have seen, this is most commonly estimated using CAPM:

$$r = R_f + \beta \cdot (E[R_m] - R_f)$$

but other models may be used.

## 1.1. Simplifying assumptions

### 1.1.1. Constant-growth model

If we assume that dividends grow at a constat rate $g$ for ever, then the valuation formula above becomes:

$$V_t = \frac{D_{t+1}}{r-g} $$

or, equivalently:


$$V_t = \frac{D_{t} \cdot (1 + g)}{r-g} $$


This is the present value of a growing perpetuity.



### 1.1.2. Multi-stage growth model

Here we estimate individually some finite number ($H$) of future dividends, and assume that after that, dividends grow at a constant rate $g$ for ever. In this case, the valuation formula becomes:

$$V_t = \frac{D_{t+1}}{1+r} + \frac{D_{t+2}}{(1+r)^2} + \frac{D_{t+3}}{(1+r)^3} + ... + \frac{D_{t+H}}{(1+r)^H} + \frac{ D_{t+H} \cdot (1 + g) / (r-g)}{(1+r)^H}$$ 

The first $H$ dividends can be estimated in a variety of ways. The most common methods include
1. Extrapolating the past dividend growth of the firm
2. Applying the average dividend growth of comparable firms
3. Applying the average dividend growth of the industry

## 1.2. Estimating the terminal growth rate

The terminal growth rate will have a significant influence on our results, so we have to be very careful when we estimate it. 
A common aproach is to use a "fundamental growth rate" (aka the "sustainable growth rate") estimate:

$$g = ROE \cdot b $$

where b is the retention ratio (aka plowback ratio):

$$b = Retention Ratio = 1 - Dividend Payout Ratio = 1 - Dividends/Net Income$$

Once again, we can use either the fundamental growth rate of the firm itself, or an average of the fundamental growth rate of close competitors, or for the industry as a whole. 

Another common approach is to assess how you think the firm will grow relative to the economy as a whole, and base your estimate of $g$ on an estimate of the perpetual growth rate in GDP (e.g. the US GDP has grown by an average of 3.1% per year in the past 70 years). 

Finally, we can use an average of past dividend growth rates, either for the firm itself or for comparable firms.

# 2. Data

### 2.1. Firm-level data

First, we use the cleaned key-ratios file for MSFT (the './msft_ratios.pkl' file created in the last lectures) to extract information on MSFT's: 
1. Dividends
2. ROE
3. Retention ratio

In [2]:
# Import data on all ratios
all_ratios = pd.read_pickle('./msft_ratios.pkl')
all_ratios

,2011-06,2012-06,2013-06,2014-06,2015-06,2016-06,2017-06,2018-06,2019-06,2020-06
Revenue USD Mil,69943.00,73723.00,77849.00,86833.00,93580.00,85320.00,89950.00,110360.00,125843.00,143015.00
Gross Margin %,77.70,76.20,74.00,69.00,64.70,61.60,61.90,65.20,65.90,67.80
Operating Income USD Mil,27161.00,27956.00,26764.00,27886.00,28172.00,21292.00,22632.00,35058.00,42959.00,52959.00
Operating Margin %,38.80,37.90,34.40,32.10,30.10,25.00,25.20,31.80,34.10,37.00
Net Income USD Mil,23150.00,16978.00,21863.00,22074.00,12193.00,16798.00,21204.00,16571.00,39240.00,44281.00
Earnings Per Share USD,2.69,2.00,2.58,2.63,1.48,2.56,3.25,2.13,5.06,5.76
Dividends USD,0.61,0.76,0.89,1.07,1.21,1.39,1.53,1.65,1.80,1.99
Payout Ratio % *,22.60,26.40,44.30,38.20,49.00,101.50,66.10,109.50,39.10,32.30
Shares Mil,8593.00,8506.00,8470.00,8399.00,8254.00,8013.00,7832.00,7794.00,7753.00,7683.00
Book Value Per Share * USD,6.82,8.19,9.21,10.61,11.23,9.58,9.05,10.32,12.41,15.12


In [3]:
# Dividends (per share)
msft_div = all_ratios.loc['Dividends USD',:]
msft_div

2011-06    0.61
2012-06    0.76
2013-06    0.89
2014-06    1.07
2015-06    1.21
2016-06    1.39
2017-06    1.53
2018-06    1.65
2019-06    1.80
2020-06    1.99
Name: Dividends USD, dtype: float64

In [4]:
# ROE
msft_roe = all_ratios.loc['Return on Equity %', :] / 100
msft_roe

2011-06    0.4484
2012-06    0.2751
2013-06    0.3009
2014-06    0.2617
2015-06    0.1436
2016-06    0.2209
2017-06    0.2937
2018-06    0.2137
2019-06    0.4241
2020-06    0.4014
Name: Return on Equity %, dtype: float64

In [5]:
# Retention ratio
msft_b = 1 - all_ratios.loc['Payout Ratio % *', :] / 100
msft_b

2011-06    0.774
2012-06    0.736
2013-06    0.557
2014-06    0.618
2015-06    0.510
2016-06   -0.015
2017-06    0.339
2018-06   -0.095
2019-06    0.609
2020-06    0.677
Name: Payout Ratio % *, dtype: float64

We also need to calculate the discount rate to use in the valuation model. This is the expected return on MSFT stock (aka the cost of equity). We use CAPM, for the purpose of this lecture, but you may use a different model (the most common alternative is the Fama-French three factor model).

We use data from 2010 onward to estimate the beta, and data from 1930 onward to estimate the market risk premium. We use a (monthly) risk free rate of 0.0001.

In [6]:
# Download the monthly Fama French factors
ff3f = pdr.DataReader('F-F_Research_Data_Factors', 'famafrench','1930-01-01', '2020-12-31')[0]/100

# Calculate risk premia
risk_premia = ff3f.mean()

# Download adj price data and drop missing values
prices = yf.download('MSFT', '2010-01-01', '2020-12-31', interval='1mo')['Adj Close'].dropna()

# Calculate returns, drop missing values and convert to dataframe
returns = prices.pct_change().dropna().to_frame()

# Rename column of returns
returns.rename(columns={'Adj Close': "MSFT"}, inplace = True)

# Convert the index to "period('M')" (monthly)
returns.index = returns.index.to_period('M')

# Combine the datasets
full_data = returns.join(ff3f)

# Run the regression to calculate CAPM beta
y = full_data['MSFT'] - full_data['RF']
X = full_data['Mkt-RF']
X = sm.add_constant(X)

# Run the CAPM regression
results = sm.OLS(y, X).fit()
print(results.summary())

# Calculate cost of equity (remember we used monthly data so we have to multiply it with 12)
msft_r =( 0.0001 + results.params['Mkt-RF'] * risk_premia['Mkt-RF']) * 12
msft_r

[*********************100%***********************]  1 of 1 completed
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.369
Model:                            OLS   Adj. R-squared:                  0.364
Method:                 Least Squares   F-statistic:                     75.43
Date:                Sat, 17 Apr 2021   Prob (F-statistic):           1.44e-14
Time:                        19:30:04   Log-Likelihood:                 210.67
No. Observations:                 131   AIC:                            -417.3
Df Residuals:                     129   BIC:                            -411.6
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------

0.07062779550417214

### 2.2. Competitor data

We obtain data on the same variables (dividends, retention ratio, ROE) for MSFT's competitors (AAPL, AMZN, GOOG) from Morningstar. However, AMZN and GOOG don't pay dividends so we use a different competitor: Oracle (ORCL). 

In general, I suggest using more than two competitors if possible, but this will suffice for the purpose of our class.

In [7]:
# Data on AAPL
aapl_div = pd.Series([0.09,0.41,0.45,0.49,0.55,0.60,0.68,0.75,0.80])
aapl_b = 1 - pd.Series([27.4,28.5,22.3,24.8,26.5,23.7,25.1,23.7,21.7]) / 100
aapl_roe = pd.Series([30.64,33.61,46.25,36.9,36.87,49.36,55.92,73.69,82.09]) / 100

#Google and Amazon don't pay dividends

# Data on ORACLE
orcl_div = pd.Series([0.21,0.24,0.30,0.48,0.51,0.60,0.64,0.76,0.81,0.96,0.96])
orcl_b = 1 - pd.Series([12.6,12.2,16.8,15.1,20.1,29.5,28.4,88.4,27.5,30.6,23.0]) / 100
orcl_roe = pd.Series([24.22,23.92,24.74,23.94,20.80,18.55,18.46,7.68,32.83,59.87,110.91]) / 100

### 2.3. Industry-level data

Please download the industry-level growth rates from D2L ("fundgr", and "histgr" files under Lectures/Data) and save them in the same folder as these lecture notes (do not change the names of these files).

These files come from Aswath Damodaran at NYU. If you want to download these data yourself, go to 

http://pages.stern.nyu.edu/~adamodar/

Then hit "Data", and from the "Current Data" menu, select "Growth/Reinvestment".

We will not import these Excel sheets into Python because we only need a single number from each of them so it's easier to just open the Excel files and look up the numbers we need.


In [8]:
# Industry fundamental growth (ROE * Retention Ratio)
msft_fg_ind = 0.198  # "fundgr.xls" file, under "Software (System & Application)"

In [9]:
# Industry historical growth rate (in net income)
msft_hg_ind = 0.1534 # "histgr.xls" file, under "Software (System & Application)"

# 3. Valuation exercises


### 3.1. Using a constant growth model

#### 3.1.1. A simple example

Assume the MSFT just paid a dividend of \\$2 per share, and that these dividends will grow at 6% per year in perpetuity. You have estimated that MSFT has a market beta of 1.1, and the market risk premium is 7% per year. The yield on a 1-year Tbill is 0.001.  What is the estimated intrinsic value of on share of MSFT?

In [10]:
# Calculate discount rate
r = 0.001 + 1.1 * 0.07
r

0.07800000000000001

In [11]:
# Estimate intrinsic value using a constant-growth DDM
v_constgr = 2 * (1 + 0.06) / (r - 0.06)
v_constgr

117.77777777777769

#### 3.1.2. Using real-world data

We have already estimated the discount rate above. We just have to estimate the terminal growth rate in dividends.
We do this using the approaches in Section 1.2.

In [12]:
# Estimate terminal growth rate using fundamental growth rates

    # Using own firm fundamental growth rate
msft_fg_own = (msft_roe * msft_b).mean()
msft_fg_own

    # Using competitor fundamental growth rate
aapl_fg = (aapl_roe * aapl_b).mean()
    
orcl_fg = (orcl_roe * orcl_b).mean()

msft_fg_comp = (aapl_fg + orcl_fg) / 2
msft_fg_comp

    # Using industry fundamental growth rate
msft_fg_ind

    # Using perpetual GDP growth as a benchmark
msft_fg_gdpc = 0.04  # slightly faster growth than historical GDP growth
msft_fg_gdpo = 0.06  # about twice as fast as historical GDP growth 

0.15580771000000002

0.31348048939393935

0.198

In [13]:
# Calculate intrinsic value using each of the fundamental growth rates used above
D0 = msft_div[-1]  # this selects the last number in the msft_div array
D0

#Using g estimated with firm's own ROE * b
price_cg_own = D0 * (1 + msft_fg_own) / (msft_r - msft_fg_own)
price_cg_own

#Using g estimated with competitors' ROE * b
price_cg_comp = D0 * (1 + msft_fg_comp) / (msft_r - msft_fg_comp)
price_cg_comp

#Using g estimated with industry's ROE * b
price_cg_ind = D0 * (1 + msft_fg_ind) / (msft_r - msft_fg_ind)
price_cg_ind

#Using g estimated using GDP Benchmark (conservative scenario)
price_cg_gdpc = D0 * (1 + msft_fg_gdpc) / (msft_r - msft_fg_gdpc)
price_cg_gdpc

#Using g estimated using GDP benchmark (optimistic scenario)
price_cg_gdpo = D0 * (1 + msft_fg_gdpo) / (msft_r - msft_fg_gdpo)
price_cg_gdpo

1.99

-27.00234388017209

-10.763010827791666

-18.716956414757583

67.57260736307572

198.4795434925253

### 3.2. Using a two-stage growth model

#### 3.2.1. A simple example

Assume the MSFT just paid a dividend of \\$2 per share, and that these dividends will grow at 30% for the next 5 years, and after that, at 6% per year in perpetuity. You have estimated that MSFT has a market beta of 1.1, and the market risk premium is 7% per year. The yield on a 1-year Tbill is 0.001.  What is the estimated intrinsic value of on share of MSFT?

In [14]:
# Inputs
D0 = 2
g1 = 0.3
g = 0.06
r = 0.001 + 1.1 * 0.07

In [15]:
# Calculate next 5 dividends
divs = pd.Series(np.nan, index = range(1,6))
divs

for t in range(1,6):
    divs[t] = D0 * ((1 + g1) **t)

divs

1   NaN
2   NaN
3   NaN
4   NaN
5   NaN
dtype: float64

1    2.60000
2    3.38000
3    4.39400
4    5.71220
5    7.42586
dtype: float64

In [16]:
# Calculate terminal value (as of 5 years from now)
p5 = divs[5] * (1 + g) / (r - g)
p5

437.3006444444442

In [17]:
# Discount dividends and terminal value to today

    # Present value of next 5 dividends
pv_div = 0
for t in range(1,6):
    pv_div = pv_div + divs[t] / ((1 + r) ** t)
pv_div

    # Present value of terminal value
pv_term = p5 / ((1+r) ** 5)
pv_term

    # Intrinsic value
v = pv_div + pv_term
v

18.158845960799553

300.3905825425361

318.54942850333566

#### 3.2.2. Using real-world data

We saw in Section 3.1.2 that the most of the fundamental growth rates we estimated for MSFT are too large to be used as perpetual growth rates. The only exceptions are the rates based on GDP growth. Below, we use these GDP-based rates as perpetual growth rates (g) and the remaining ones as intermediate growth rates (similar to g1 in 3.2.1). We also add average past dividend growth rates as estimates for the intermediate growth rate.

We will use a horizon (H) for the intermediate growth rate and the perpetual growth value thereafter. 

In [18]:
# Estimate intermediate growth rate based on average past dividend growth rates

    # Own firm average dividend growth rate
msft_dg_own = msft_div.pct_change().mean()
msft_dg_own

    # Comps average div growth rate
aapl_dg = aapl_div.pct_change().mean()
aapl_dg

orcl_dg = orcl_div.pct_change().mean()
orcl_dg

msft_dg_comp = (aapl_dg + orcl_dg) / 2
msft_dg_comp

    # Industry average div growth rate (assume it matches earnings growth rate)
msft_dg_ind = msft_hg_ind
msft_dg_ind

0.14160203972026475

0.5322880833782144

0.17369690566284995

0.35299249452053216

0.1534

In [19]:
# Calculate future 10 dividends, based on each of the 6 estimated growth rates
    
    # Latest dividend
D0 = msft_div[-1]
D0

    # Create empty dataframe to store all our results
model_names = ['fg_own','fg_comp','fg_ind','dg_own','dg_comp','dg_ind']    
divs = pd.DataFrame(np.nan, index = range(1,11), columns=model_names)
divs

    # Calculate future dividends
for t in range(1,11):
    divs.loc[t,'fg_own'] = D0 * (1+msft_fg_own)**t
    divs.loc[t,'fg_comp'] = D0 * (1+msft_fg_comp)**t
    divs.loc[t,'fg_ind'] = D0 * (1+msft_fg_ind)**t
    divs.loc[t,'dg_own'] = D0 * (1+msft_dg_own)**t
    divs.loc[t,'dg_comp'] = D0 * (1+msft_dg_comp)**t
    divs.loc[t,'dg_ind'] = D0 * (1+msft_dg_ind)**t


divs    

1.99

,fg_own,fg_comp,fg_ind,dg_own,dg_comp,dg_ind
1,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN
10,NaN,NaN,NaN,NaN,NaN,NaN


,fg_own,fg_comp,fg_ind,dg_own,dg_comp,dg_ind
1,2.300057,2.613826,2.384020,2.271788,2.692455,2.295266
2,2.658424,3.433210,2.856056,2.593478,3.642871,2.647360
3,3.072627,4.509454,3.421555,2.960720,4.928778,3.053465
4,3.551366,5.923080,4.099023,3.379964,6.668599,3.521866
5,4.104696,7.779850,4.910629,3.858573,9.022565,4.062121
6,4.744239,10.218681,5.882934,4.404955,12.207463,4.685250
7,5.483429,13.422038,7.047755,5.028706,16.516605,5.403967
8,6.337789,17.629585,8.443211,5.740781,22.346843,6.232936
9,7.325265,23.156116,10.114966,6.553687,30.235111,7.189068
10,8.466598,30.415106,12.117730,7.481703,40.907878,8.291871


In [20]:
# Present value of dividends
    # Create dataframe to store results
pv_divs = pd.DataFrame(0, index = [10], columns = model_names)
pv_divs

    # Calculate present value of future dividends for each model
for m in model_names:
    for t in range(1,11):
        pv_divs.loc[10, m] = pv_divs.loc[10, m] + divs.loc[t, m] / ((1+msft_r)**t)

pv_divs

,fg_own,fg_comp,fg_ind,dg_own,dg_comp,dg_ind
10,0,0,0,0,0,0


,fg_own,fg_comp,fg_ind,dg_own,dg_comp,dg_ind
10,31.057009,72.372179,38.882484,28.80897,89.526551,30.663362


In [21]:
# Use optimistic gdp-growth based perpetual growth rate to estimate terminal value 10 years from now
    # Perpetual growth rate
g = msft_fg_gdpo
g

    # Create dataframe to store results
p10 = pd.DataFrame(np.nan, index = [10], columns = model_names)
p10

    # Calculate terminal value as of 10 years from now
for m in model_names:
    p10.loc[10, m] = divs.loc[10, m] * (1 + g) / (msft_r - g)

p10

0.06

,fg_own,fg_comp,fg_ind,dg_own,dg_comp,dg_ind
10,NaN,NaN,NaN,NaN,NaN,NaN


,fg_own,fg_comp,fg_ind,dg_own,dg_comp,dg_ind
10,844.445497,3033.555978,1208.603737,746.213519,4080.088908,827.018497


In [22]:
# Present value of terminal value
pv_term = p10 / ((1 + msft_r) ** 10)
pv_term

,fg_own,fg_comp,fg_ind,dg_own,dg_comp,dg_ind
10,426.762727,1533.087245,610.799665,377.118615,2061.980167,417.955534


In [23]:
# Intrinsic value
v = pv_divs + pv_term
v

,fg_own,fg_comp,fg_ind,dg_own,dg_comp,dg_ind
10,457.819736,1605.459425,649.682149,405.927585,2151.506718,448.618896
